# MQTT

In this exercise you will learn:
- How to use the MQTT client.
- How to use the Mosquitto MQTT server.

## Install the Mosquitto MQTT Server

1. Use [this link](https://mosquitto.org/download) to downlaod and install the Mosquitto MQTT Server.
2. Make sure the mosquitto executable file is on your path.
3. Make sure to [stop the Mosquitto MQTT service](http://www.steves-internet-guide.com/mosquitto-broker/#:~:text=On%20Windows%20you%20can%20stop,control%20panel%3Eadmin%3Eservices.&text=By%20default%20the%20broker%20will%20start%20listening%20on%20port%201883.).

You can start the Mosquitto MQTT Server from the command prompt with `mosquitto -v`.  
You can stop the Mosquitto MQTT Server from the command prompt with `<ctrl> + <c>`.

The Mosquitto MQTT server uses port `1883` as default (but can be changed with the flag `-p <port>`).

As a first step, let's import the python modules we need.

In [1]:
import paho.mqtt.client as mqtt # pip install paho.mqtt
import time

## The MQTT Architecture

As default, the MQTT client is used to communicate with a MQTT Server running a MQTT Broker service.

Let's look at a simple scenario, where we have:

1. One device (say a Raspberry Pi), with one temperature sensor connected to it, running an MQTT client.
2. One device (say an Android or iOS mobile phone) running an MQTT client.
3. One device (say a Web Server) running an MQTT client.
4. One device (say another Raspberry Pi) running an MQTT Broker on an MQTT Server.

Device 1 (MQTT client) samples the temperature from the attached temperature sensor (with a certain sampling frequency) and sends this reading to the MQTT Broker running on the MQTT server (Device 4).

The MQTT Broker on Device 4 (MQTT server) receives messages (temperature readings) from Device 1 (MQTT client), and forwards them to Device 2 and 3 (MQTT clients).

Device 2 and 3 (MQTT clients) receive messages (temperature readings) from the MQTT Broker on the MQTT server (Device 4).


<img width="800" height="300" src="../../notebook_images/mqtt-publish-subscribe.png">

## The Publish/Subscribe Pattern

The Publish/Subscribe Pattern has Publishers and Subscribers:
- A **publisher** *publishes* **messages** to a specific **topic**.
  - A **topic** can be compared with a mail box with a unique address.
  - A **topic** is simply a string with a unique value, e.g. "building1/room2/temperature3".
  - A **topic** is unique for a specific MQTT Broker (i.e. compare the broker with a post office containing the mail boxes).
  - A **message** *published* to a specific **topic** ends up in the unqiue mail box on the MQTT Broker.
- A **subscriber** *subscribes* to a specific **topic**.
  - Any **message** *published* by a **publisher** to a specific **topic**, is sent to all **subscribers** *subscribed* to that **topic**.

In our example scenario above:

1. All MQTT clients (Device 1, 2 and 3) connect to the MQTT server (Device 4) running the MQTT broker.
2. Device 2 and 3 (MQTT clients) subscribe to the topic "temperature" on the MQTT broker running on the MQTT server (Device 4).
3. Device 1 (MQTT client) publishes temperature readings (messages) to topic "temperature".
4. The MQTT broker on Device 4 (MQTT server) receives the message for topic "temperature" from Device 1 (MQTT client).
5. The MQTT broker on Device 4 (MQTT server) sends the message for topic "temperature" to MQTT clients subscribed to topic "temperature" (Device 2 and 3).

## The MQTT Client Message Loop

<img width="500" height="300" src="../../notebook_images/loop-function-illustration.jpg" style="float: right;">

Each MQTT Client has a **Message Loop** that runs on a separate background thread and is responsible for:
- Sending messages to the MQTT broker on the MQTT server.
- Receiving messages from the MQTT broker on the MQTT server.

When an MQTT client publishes a message to a specific topic:

1. The message is queued in an **outgoing message buffer**.
2. The message loop *pops* the next message from the **outgoing message buffer** and sends it to the MQTT broker on the MQTT server.

When an MQTT client receives message from the MQTT boker for a specific topic:

1. The message is queued in an **incoming message buffer**.
2. The message loop *pops* the next message from the **incoming message buffer** and **raises an on_message event**.
3. The MQTT client **handles** the message via its **on_message event handler** (callback function).

## Simple Example

Let's look at a very simple example, where:
- We have one MQTT client that publishes and subscribes to the same topic.
- We have an MQTT broker running as a service on a MQTT server in the cloud.
  - The Mosquitto MQTT cloud server "test.mosquitto.org" can be used during development.

Let's start be creating our **on_message** callback function (event handler) to receive incoming message events from the message loop.

The **on_message** callback function (event handler) expects three input parameters:
- `client` represents the client that sent the message.
- `userdata` is any additional data about the client that sent the message (set by the client).
- `message` is the received message.

We are mostly interested in the `message` parameter in this example, which has the properties:
- `payload` contains the actual message.
- `topic` contains the topic for the message.
- `qos` contains the level of quality of service for the message.
- `retain` contains the value of a retain flag for the message.

We are mostly interested in the `payload` in this example:
- To extract the message, we use: `message.payload.decode('utf-8')`

In [2]:
# Create an "on_message" callback function (event handler) for the "on_message" event
def on_message(client, userdata, message):
    print(f"\nmessage payload: {message.payload.decode('utf-8')}")
    print(f"message topic: {message.topic}")
    print(f"message qos: {message.qos}")
    print(f"message retain flag: {message.retain}")

Next we will:
- Create an instance of a MQTT client.
- Attach our **on_message** callback function (event handler) to the client instance's **on_message** property.

The MQTT Client class has a constructor with the following default parameters:
- `client_id = ''`
- `clean_session = None`
- `userdata = None`
- `protocol = 4`
- `transport = 'tcp'`

We are mostly interested in the `client_id` in this example, which we will use to set an ID for the client (that the MQTT broker can use to identify the MQTT client).

In [3]:
print("creating new instance")
client = mqtt.Client("P1")     # create new instance (the ID, in this case "P1", must be unique)
client.on_message = on_message # attach "on_message" callback function (event handler) to "on_message" event

creating new instance


Once we have a MQTT client instance, with an **on_message** callback function (event handler) attached, we can use the MQTT client's following methods (functions):
- `connect(broker_address)` to connect to the MQTT broker running on the MQTT server with address `broker_address` (IP or DNS).
- `loop_start()` to start the **message loop**.
- `subscribe(topic)` to subscribe to the topic `topic`.
- `publish(topic, message)` to publish the message (payload) `message` to the topic `topic`.
- `unsubscribe(topic)` to unsubscribe from the topic `topic`.
- `loop_stop()` to stop the **message loop**.
- `disconnect()` to disconnect from the MQTT broker on the MQTT server.

Note:
- Some of the functions above accept additional input parameters (but we will use the default values in this example).
- The `broker_address` can be an IP Address or DNS name (domain name).
- We can set `broker_address` to:
  - `test.mosquitto.org` in order to use the test MQTT cloud server for development purposes.
  - To the IP or DNS of any other machine running an MQTT server, e.g. the [Mosquitto MQTT server](https://mosquitto.org/download).

In [4]:
#broker_address = "localhost" # Use your own MQTT Server IP Adress (or domain name) here, or ...
broker_address = "test.mosquitto.org" # ... use the Mosquitto test server during development

# Use exception handling (try...except in Python)
try:
    print("connecting to broker")
    client.connect(broker_address) # connect to broker
    client.loop_start()            # start the event processing loop

    print("Subscribing to topic: house/bulbs/bulb1")
    client.subscribe("house/bulbs/bulb1") # subscribe

    print("Publishing message 'OFF' to topic: house/bulbs/bulb1")
    client.publish("house/bulbs/bulb1", "OFF") # publish

    print("Unsubscribing from topic: house/bulbs/bulb1")
    client.unsubscribe("house/bulbs/bulb1") # unsubscribe

    time.sleep(4)       # wait 4 seconds before stopping the event processing loop (so all pending events are processed)
    client.loop_stop()  # stop the event processing loop

    print("\ndisconnecting from broker")
    client.disconnect() # disconnect from broker
except Exception as e:
    # if we receive an exception (error) in the "try" block,
    # handle it here, by printing out the error message
    print(f"connection error: {e}")

connecting to broker
Subscribing to topic: house/bulbs/bulb1
Publishing message 'OFF' to topic: house/bulbs/bulb1
Unsubscribing from topic: house/bulbs/bulb1

message payload: OFF
message topic: house/bulbs/bulb1
message qos: 0
message retain flag: 0

disconnecting from broker


If you want to download and try running the [Mosquitto MQTT server](https://mosquitto.org/download) on your own machine (instead of connecting to the cloud server "test.mosquitto.org"), start the Mosquitto MQTT server from the command line with the following command (the `-v` flag means "verbose"):

`mosquitto -v`

As default the Mosquitto server listens on port `1883`, which you can change using the flag `-p <port>`, e.g.:

`mosquitto -v -p 8080`

In that case, your MQTT client needs to connect to your own computer's IP Address (or host name) using that port, e.g.:

`client.connect(ip_or_host, port=8080)`

If you need to find you computer's Hostname and/or local IP Address, your can use this Python code:

```python
import socket

hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)

print(hostname)
print(local_ip)
```

You can also use the `ipconfig` (Windows) or `ifconfig` (Linux, MacOS) commands in the terminal.

The easiest is probably just to connect to `localhost` during development when running Mosquitto on your own computer.

Note that the Mosquitto executable `mosquitto` needs to be on your PATH, and you need to stop the service for the Mosquitto server (the installation program configures Mosquitto to run as a service by default), otherwise you won't see any print-outs from the Mosquitto server in your terminal.

For more information about the MQTT client and Mosquitto MQTT server, see:
- http://www.steves-internet-guide.com/into-mqtt-python-client
- https://mosquitto.org